## Comparing iperf data to speedtest data

In [1]:
from data_exploration import *
import numpy as np

Since iperf data stopped coming at December 3rd 2018, we will choose December 4rd 2018 00:00 as a starting point and will select all data collected by iperf and speedtes back from this date:

In [2]:
#Set up starting point, by default if will start from current time
#starting_point=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
starting_point="2018-12-04 00:00:00"  # to set upl alternative starting point
print("Starting point:",starting_point )

title_tail=" to the date "+ starting_point
query_tail=" AND time < '"+starting_point+"'"

Starting point: 2018-12-04 00:00:00


In [3]:
#Set up influxdb connection:
client, client_df = connect_to_influxdb()

### Statistics by device

#### Download speed

In [4]:
query_download1 = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER!='iperf' AND DOWNLOAD>0"+ query_tail+";"
download_df1 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download1,table_name='SPEEDTEST_DOWNLOAD')
device_numbers_d1=download_df1['SK_PI'].unique()
device_numbers_d1=list(map(int, device_numbers_d1))
device_numbers_d1= sorted(device_numbers_d1)

In [5]:
query_download2 = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER='iperf' AND DOWNLOAD>0"+ query_tail+";"
download_df2 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download2,table_name='SPEEDTEST_DOWNLOAD')
download_df2['DOWNLOAD']=download_df2['DOWNLOAD']*0.001
device_numbers_d2=download_df2['SK_PI'].unique()
device_numbers_d2=list(map(int, device_numbers_d2))
device_numbers_d2= sorted(device_numbers_d2)

In [6]:
result1=pd.DataFrame(download_df1.groupby('SK_PI').size())
result1.columns=['size1']

result2=pd.DataFrame(download_df2.groupby('SK_PI').size())
result2.columns=['size2']
result=result2.join(result1,how='outer')
#result

In [7]:
combined_bar_plot_2traces(xvalues=result.index.astype(int),
                          yvalues1=result['size1'],
                          yvalues2=result['size2'],
                          name1='speedtest',
                          name2='iperf',
                          title="Comparing number of datapoints for speedtest and iperf"+title_tail ,
                          ytitle="Number of datapoints")

In [8]:
t="Download speed by hour for speedtest VS iperf data"+title_tail
scatterplot_2groups(title=t,dataframe1=download_df1,dataframe2=download_df2,
                    plot_value="DOWNLOAD",ytitle="Download speed (Mbps)")

In [9]:
boxplot_2groups(dataframe1=download_df1,dataframe2=download_df2,plot_value='DOWNLOAD',sort_value='SK_PI',
               title="Download speed by device (orange - speedtest, purple - iperf)"+title_tail,
               ytitle="Mbps", xtitle="Device number", downloadline=True)

In [10]:
result11=pd.DataFrame(download_df1.groupby('SK_PI')['DOWNLOAD'].mean())
result11.columns=['mean1']
result12=pd.DataFrame(download_df1.groupby('SK_PI')['DOWNLOAD'].median())
result12.columns=['median1']
result1=result11.join(result12,how='outer')

result21=pd.DataFrame(download_df2.groupby('SK_PI')['DOWNLOAD'].mean())
result21.columns=['mean2']
result22=pd.DataFrame(download_df2.groupby('SK_PI')['DOWNLOAD'].median())
result22.columns=['median2']
result2=result21.join(result22,how='outer')

result=result2.join(result1,how='outer')
#result

In [11]:
device_numbers_d=result.index
download_line=go.Scatter(x=device_numbers_d,y=[50] * len(device_numbers_d), mode='markers',marker=dict(color='red'), name='50Mbps')

combined_bar_plot_4traces(xvalues=result.index,
                         yvalues1=result["mean1"],
                         yvalues2=result["mean2"],
                         yvalues3=result["median1"],
                         yvalues4=result["median2"],
                         name1="Mean speedtest",
                         name2="Mean iperf",
                         name3="Median speedtest",
                         name4="Median iperf",
                         title="Download speed by device" +title_tail,
                         ytitle="Mbps",
                         line=download_line,
                         stack=False)

#### Upload speed

In [12]:
query_upload1 = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0"+ query_tail+";"
upload_df1 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload1,table_name='SPEEDTEST_UPLOAD')
device_numbers_u1=upload_df1['SK_PI'].unique()
device_numbers_u1=list(map(int, device_numbers_u1))
device_numbers_u1= sorted(device_numbers_u1)

In [13]:
query_upload2 = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER='iperf' AND UPLOAD>0"+ query_tail+";"
upload_df2 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload2,table_name='SPEEDTEST_UPLOAD')
upload_df2['UPLOAD']=upload_df2['UPLOAD']*0.001
device_numbers_u2=download_df2['SK_PI'].unique()
device_numbers_u2=list(map(int, device_numbers_u2))
device_numbers_u2= sorted(device_numbers_u2)

In [14]:
t="Upload speed by hour for speedtest VS iperf data"+title_tail
scatterplot_2groups(title=t,dataframe1=upload_df1,dataframe2=upload_df2,
                    plot_value="UPLOAD",ytitle="Upload speed (Mbps)")

In [15]:
boxplot_2groups(dataframe1=upload_df1,dataframe2=upload_df2,plot_value='UPLOAD',sort_value='SK_PI',
               title="Upload speed by device (orange - speedtest, purple - iperf)"+title_tail,
               ytitle="Mbps", xtitle="Device number", uploadline=True)

In [16]:
result11=pd.DataFrame(upload_df1.groupby('SK_PI')['UPLOAD'].mean())
result11.columns=['mean1']
result12=pd.DataFrame(upload_df1.groupby('SK_PI')['UPLOAD'].median())
result12.columns=['median1']
result1=result11.join(result12,how='outer')

result21=pd.DataFrame(upload_df2.groupby('SK_PI')['UPLOAD'].mean())
result21.columns=['mean2']
result22=pd.DataFrame(upload_df2.groupby('SK_PI')['UPLOAD'].median())
result22.columns=['median2']
result2=result21.join(result22,how='outer')

result=result2.join(result1,how='outer')
#result

In [17]:
device_numbers_u=result.index
upload_line=go.Scatter(x=device_numbers_d,y=[10] * len(device_numbers_u), mode='markers',marker=dict(color='red'), name='10Mbps')

combined_bar_plot_4traces(xvalues=result.index,
                         yvalues1=result["mean1"],
                         yvalues2=result["mean2"],
                         yvalues3=result["median1"],
                         yvalues4=result["median2"],
                         name1="Mean speedtest",
                         name2="Mean iperf",
                         name3="Median speedtest",
                         name4="Median iperf",
                         title="Upload speed by device" +title_tail,
                         ytitle="Mbps",
                         line=upload_line,
                         stack=False)

### Ping latency

In [24]:
query_ping1 = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER!='iperf' AND PING>0"+ query_tail+";"
ping_df1 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping1,table_name='SPEEDTEST_PING')
device_numbers_p1=ping_df1['SK_PI'].unique()
device_numbers_p1=list(map(int, device_numbers_p1))
device_numbers_p1= sorted(device_numbers_p1)

In [26]:
query_ping2 = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER='iperf' AND PING>0"+ query_tail+";"
ping_df2 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping2,table_name='SPEEDTEST_PING')
device_numbers_p2=ping_df2['SK_PI'].unique()
device_numbers_p2=list(map(int, device_numbers_p2))
device_numbers_p2= sorted(device_numbers_p2)

In [28]:
t="Ping latency by hour for speedtest VS iperf data"+title_tail
scatterplot_2groups(title=t,dataframe1=ping_df1,dataframe2=ping_df2,
                    plot_value="PING",ytitle="Ping latency(Miliseconds)")

In [30]:
boxplot_2groups(dataframe1=ping_df1,dataframe2=ping_df2,plot_value='PING',sort_value='SK_PI',
               title="Ping latency by device (orange - speedtest, purple - iperf)"+title_tail,
               ytitle="Miliseconds", xtitle="Device number")

In [31]:
result11=pd.DataFrame(ping_df1.groupby('SK_PI')['PING'].mean())
result11.columns=['mean1']
result12=pd.DataFrame(ping_df1.groupby('SK_PI')['PING'].median())
result12.columns=['median1']
result1=result11.join(result12,how='outer')

result21=pd.DataFrame(ping_df2.groupby('SK_PI')['PING'].mean())
result21.columns=['mean2']
result22=pd.DataFrame(ping_df2.groupby('SK_PI')['PING'].median())
result22.columns=['median2']
result2=result21.join(result22,how='outer')

result=result2.join(result1,how='outer')
#result

In [33]:
combined_bar_plot_4traces(xvalues=result.index,
                         yvalues1=result["mean1"],
                         yvalues2=result["mean2"],
                         yvalues3=result["median1"],
                         yvalues4=result["median2"],
                         name1="Mean speedtest",
                         name2="Mean iperf",
                         name3="Median speedtest",
                         name4="Median iperf",
                         title="Ping latency by device" +title_tail,
                         ytitle="Miliseconds",
                         stack=False)